In [137]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import random
import tensorflow as tf
import csv

import tensorflow as tf
from torch.utils.data import TensorDataset, DataLoader
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.applications import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [138]:
# housing_csv = pd.read_csv('../data/housing.csv')
new_housing_csv = pd.read_csv('../data/new_housing.csv')

#Length
print(len(new_housing_csv))

#Mask Sale Price
mask = (new_housing_csv["SALE PRICE"]) > 100000.0
new_housing_csv = new_housing_csv[mask]


#Mask for Sold after 2010
new_housing_csv['SALE DATE'] = pd.to_datetime(new_housing_csv['SALE DATE'])
mask = (new_housing_csv["SALE DATE"]).dt.year > 2011 
new_housing_csv = new_housing_csv[mask]
mask = (new_housing_csv["SALE DATE"]).dt.year < 2015
new_housing_csv = new_housing_csv[mask]

#Length
print(len(new_housing_csv))

#Only important rows
new_housing_csv = new_housing_csv[["LAND VAL", "PARCEL VAL", "IMPROVE VAL", "SALE PRICE", "SQFT", "ROOMS", "BEDROOM", "BATH", "LIVING AREA", "GROSS AREA", "YEAR", "PERCENT GOOD", "STORIES"]]


#Remove Nulls
mask = new_housing_csv.notnull().all(axis=1)
new_housing_csv = new_housing_csv.loc[mask, :]
print(len(new_housing_csv))

#Remove 0s
mask = (new_housing_csv != 0).all(axis=1)
new_housing_csv = new_housing_csv.loc[mask, :]


#Length
print(len(new_housing_csv))


housing_data = new_housing_csv[["LAND VAL", "PARCEL VAL", "SQFT", "ROOMS", "BEDROOM", "BATH", "LIVING AREA", "GROSS AREA", "YEAR","PERCENT GOOD", "STORIES"]]
housing_label = new_housing_csv[["SALE PRICE"]]



# print(new_housing_csv)

# housing_data = housing_csv[["housing_median_age", "total_rooms", "total_bedrooms", "median_income", "population", "households"]]
# housing_label = housing_csv[["median_house_value"]]


107899
10582
10224
3601


/var/folders/p6/bmvt9pvd0vd5t3yfr1yhfh580000gn/T/ipykernel_87136/713889739.py:2: DtypeWarning: Columns (13,20,53,54,59,60,61,62,63,64,65,66,67,68,69,70) have mixed types. Specify dtype option on import or set low_memory=False.
  new_housing_csv = pd.read_csv('../data/new_housing.csv')


In [139]:
X = housing_data
Y = housing_label

In [140]:
mu = np.mean(X, axis=0)
sigma = np.std(X, axis=0)
X = (X - mu) / sigma

mu = np.mean(Y, axis=0)
sigma = np.std(Y, axis=0)
Y = (Y - mu) / sigma

X = pd.DataFrame.to_numpy(X)
Y = pd.DataFrame.to_numpy(Y)


In [141]:
class LinearRegression(tf.keras.Model):
    def __init__(self):
        super(LinearRegression, self).__init__()
        self.dense1 = tf.keras.layers.Dense(units=128, activation='relu', input_shape=[1])
        self.dense2 = tf.keras.layers.Dense(units=64, activation='relu')
        self.dense3 = tf.keras.layers.Dense(units=32, activation='relu')
        self.dense4 = tf.keras.layers.Dense(units=1, kernel_regularizer=tf.keras.regularizers.l2(0.01))
        
    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        return x

In [142]:
model = LinearRegression()

# Define the loss function and optimizer
loss_fn = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.015)

In [143]:
x = tf.convert_to_tensor(X, dtype=tf.float32)
y = tf.convert_to_tensor(Y, dtype=tf.float32)


print(len(x))
x_train = x[:3000]
y_train = y[:3000]
x_test = x[3001:3601]
y_test = y[3001:3601]
# Remove all instances where data is empty

3601


In [ ]:
num_epochs = 10000
for epoch in range(num_epochs):
    # Forward pass
    with tf.GradientTape() as tape:
        y_pred = model(x_train)
        loss = loss_fn(y_train, y_pred)
    
    # Backward and optimize
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
    # Print progress
    if (epoch+1) % 100 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.numpy():.4f}")

Epoch [100/10000], Loss: 0.8882
Epoch [200/10000], Loss: 0.8018
Epoch [300/10000], Loss: 0.7522
Epoch [400/10000], Loss: 0.7273
Epoch [500/10000], Loss: 0.7127
Epoch [600/10000], Loss: 0.6967
Epoch [700/10000], Loss: 0.6883
Epoch [800/10000], Loss: 0.6832
Epoch [900/10000], Loss: 0.6794
Epoch [1000/10000], Loss: 0.6763
Epoch [1100/10000], Loss: 0.6737
Epoch [1200/10000], Loss: 0.6716
Epoch [1300/10000], Loss: 0.6696
Epoch [1400/10000], Loss: 0.6679
Epoch [1500/10000], Loss: 0.6663
Epoch [1600/10000], Loss: 0.6648
Epoch [1700/10000], Loss: 0.6631
Epoch [1800/10000], Loss: 0.6524
Epoch [1900/10000], Loss: 0.6382
Epoch [2000/10000], Loss: 0.6331
Epoch [2100/10000], Loss: 0.6307
Epoch [2200/10000], Loss: 0.6291
Epoch [2300/10000], Loss: 0.6277
Epoch [2400/10000], Loss: 0.6264


In [ ]:
print(model.layers[0].weights)

In [ ]:
def accuracy(y_pred, y_true, sigma, mu, threshold=0.2):
    """
    Computes the accuracy of the predictions given the true labels, based on a threshold value.
    
    Parameters:
    y_pred (np.ndarray): An array of predicted labels.
    y_true (np.ndarray): An array of true labels.
    threshold (float): The threshold value to use for measuring accuracy.
    
    Returns:
    float: The accuracy of the predictions as a percentage.
    """
    # Compute the absolute difference between predicted and true values
    
    
    y_pred = (y_pred * sigma) + mu
    y_true = (y_true * sigma) + mu

    diff = np.abs(y_pred - y_true)
    print(y_pred)
    print(y_true)
    
    # Compute accuracy
    acc = np.mean(diff <= threshold * y_pred)
    
    # Convert to percentage
    acc_pct = acc * 100
    
    return acc_pct

In [ ]:
pred = model(x_test)
loss = loss_fn(y_test, pred)
accuracy(pred, y_test, sigma, mu)

In [ ]:
weights, bias = model.layers[0].get_weights()
print(weights)
